In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
from scipy.stats import norm
from scipy.stats import poisson
import statsmodels.formula.api as sm
import warnings
from os import path as path
import math
warnings.filterwarnings('ignore')

def LLR2p(LLR):
    LR = np.exp(LLR)
    return (LR)/(1+LR)

def LLR2pcorrect(LLR):
    return (LLR2p(np.abs(LLR)))

### Discrimination

In [2]:
class DiscriminationModel:
    def __init__(self, mu, sigma, perceptual_noise):
        
        self.df = pd.DataFrame()
        self.mu = mu
        self.sigma = sigma
        self.perceptual_noise = perceptual_noise
        
        self.signal_dist = stats.norm(self.mu[1],np.sqrt(self.sigma[1]**+self.perceptual_noise**2));
        self.noise_dist = stats.norm(self.mu[0],np.sqrt(self.sigma[0]**+self.perceptual_noise**2));

    def runModel(self, num_trials, num_timepoints):
        
        self.num_trials = num_trials;
        self.num_timepoints = num_timepoints;
        
        self.df['trial_id'] = range(num_trials);
        
        # first, decide which is the true direction in each trial (p=0.5)
        self.df['bright_side'] = [1 if flip else 0 
                                for flip in np.random.binomial(1,0.5,num_trials)] 
        
        # duplicate each row num_timepoints times
        self.df = self.df.loc[self.df.index.repeat(num_timepoints)].reset_index(drop=True)

        self.df['timepoint'] = list(range(1,num_timepoints+1))*num_trials;
        
        # duplicate each row twice (for the two evidence channels)
        self.df = self.df.loc[self.df.index.repeat(2)].reset_index(drop=True)
        
        self.df['side'] = [1,0] * num_trials*num_timepoints;
        
        self.getMotionEnergy()
        
        self.extractLLR()

        # I didn't have a better word for this part, which includes processing
        # the stimuli, making a decision and rating confidence. 
        self.behave()
        
        self.df['correct'] = self.df.apply(lambda row: 
                                           row.bright_side==row.decision, axis=1)
        
    def getMotionEnergy(self):
        
        # sample the motion energy for left and right as a function of the true direction
        self.df['evidence'] = self.df.apply(lambda row: 
                      np.random.normal(self.mu[1],self.sigma[1]) 
                      if row.side==row.bright_side
                      else np.random.normal(self.mu[0],self.sigma[0]),
                      axis=1)

        # how it appears to subjects
        self.df['percept'] = self.df.apply(lambda row: row.evidence +
                        np.random.normal(0, self.perceptual_noise), 
                        axis=1);
    
    def extractLLR(self):
        
        # extract the Log Likelihood Ratio (LLR) 
        #log(p(evidence|signal))-log(p(evidence|noise))
        self.df['sample_LLR'] = self.df.apply(lambda row: self.signal_dist.logpdf(row.percept)-
                                              self.noise_dist.logpdf(row.percept), axis=1)
        
    def behave(self):
        
        exp_LLR = [];
        exp_decision = [];
        exp_confidence = [];

        trials=self.df.trial_id.unique()

        for i_trial in range(self.num_trials):
            trial_df = self.df[self.df.trial_id==i_trial]
            trial_LLR = [0,0];

            LLR = 0;
            
            for i_timepoint in range(2,self.num_timepoints+1):

                sample1 = trial_df[(trial_df.timepoint==i_timepoint) & 
                                   (trial_df.side==1)]['sample_LLR'].values[0];
                sample0 = trial_df[(trial_df.timepoint==i_timepoint) & 
                                   (trial_df.side==0)]['sample_LLR'].values[0];
                LLR = LLR+sample1-sample0;

                trial_LLR += [LLR,LLR]

            trial_decision = [int(LLR>0)]*24
            trial_confidence = [LLR2pcorrect(LLR)]*24     

            exp_LLR = exp_LLR + trial_LLR;
            exp_decision = exp_decision+trial_decision;
            exp_confidence = exp_confidence+trial_confidence;

        self.df['LLR'] = exp_LLR;
        self.df['decision'] = exp_decision;
        self.df['confidence'] = exp_confidence;
        
    def save(self,file_name):
        self.df.to_csv(file_name);

In [3]:
np.random.seed(1)

uv=DiscriminationModel([0,0.5],[1,1],2)
uv.runModel(10000,12)
uv.save(path.join(
        '..','simulations','equal_variance','discrimination.csv'))

FileNotFoundError: [Errno 2] No such file or directory: '..\\simulations\\equal_variance\\discrimination.csv'

### Detection

In [5]:
class DetectionModel:
    def __init__(self, mu, sigma, perceptual_noise):
        
        self.df = pd.DataFrame()
        self.mu = mu
        self.sigma = sigma
        self.perceptual_noise= perceptual_noise;
        
        self.signal_dist = stats.norm(self.mu[1],np.sqrt(self.sigma[1]**+self.perceptual_noise**2));
        self.noise_dist = stats.norm(self.mu[0],np.sqrt(self.sigma[0]**+self.perceptual_noise**2));
      
    def runModel(self, num_trials, num_timepoints):
        
        self.num_trials = num_trials;
        self.num_timepoints = num_timepoints;
        
        self.df['trial_id'] = range(num_trials);
        
        # first, decide which is the true direction in each trial (p=0.5)
        self.df['bright_side'] = [1 if flip else 0 
                                for flip in np.random.binomial(1,0.5,num_trials)] 
        
        # first, decide which is the true direction in each trial (p=0.5)
        self.df['signal'] = [1 if flip else 0 
                                for flip in np.random.binomial(1,0.5,num_trials)] 
        
        # duplicate each row num_timepoints times
        self.df = self.df.loc[self.df.index.repeat(num_timepoints)].reset_index(drop=True)

        self.df['timepoint'] = list(range(1,num_timepoints+1))*num_trials;
        
        # duplicate each row twice (for the two evidence channels)
        self.df = self.df.loc[self.df.index.repeat(2)].reset_index(drop=True)
        
        self.df['side'] = [1,0] * num_trials*num_timepoints;
        
        self.getMotionEnergy()
        
        self.extractLLR()

        # I didn't have a better word for this part, which includes processing
        # the stimuli, making a decision and rating confidence. 
        self.behave()
        
        self.df['correct'] = self.df.apply(lambda row: 
                                           row.signal==row.decision, axis=1)
        
    def getMotionEnergy(self):
        
        # sample the motion energy for left and right as a function of the true direction
        self.df['evidence'] = self.df.apply(lambda row: 
                      np.random.normal(self.mu[1],self.sigma[1]) 
                      if (row.side==row.bright_side) & (row.signal)==1
                      else np.random.normal(self.mu[0],self.sigma[0]),
                      axis=1)

        # how it appears to subjects
        self.df['percept'] = self.df.apply(lambda row: row.evidence +
                        np.random.normal(0, self.perceptual_noise), 
                        axis=1);
    
    def extractLLR(self):
        
        self.df['likelihood_signal'] = self.df.apply(lambda row: self.signal_dist.pdf(row.percept), axis=1);
        self.df['likelihood_noise'] = self.df.apply(lambda row: self.noise_dist.pdf(row.percept), axis=1)
        
        # extract the Log Likelihood Ratio (LLR) 
        #log(p(evidence|signal))-log(p(evidence|noise))
        self.df['sample_LLR'] = self.df.apply(lambda row: self.signal_dist.logpdf(row.percept)-
                                              self.noise_dist.logpdf(row.percept), axis=1)
        
    def behave(self):
        
        exp_LLR = [];
        exp_decision = [];
        exp_confidence = [];

        trials=self.df.trial_id.unique()

        for i_trial in range(self.num_trials):
            trial_df = self.df[self.df.trial_id==i_trial]
            trial_LLR = [0,0];

            LLR1 = 0; # likelihood ratio regarding the two side of the signal, conditioned on that there is a signal
            LLR_present = 0;
            
            for i_timepoint in range(2,self.num_timepoints+1):

                sample1 = trial_df[(trial_df.timepoint==i_timepoint) & 
                                   (trial_df.side==1)]['sample_LLR'].values[0];
                sample0 = trial_df[(trial_df.timepoint==i_timepoint) & 
                                   (trial_df.side==0)]['sample_LLR'].values[0];
                
                psignal1 = trial_df[(trial_df.timepoint==i_timepoint) & 
                                   (trial_df.side==1)]['likelihood_signal'].values[0];
                psignal0 = trial_df[(trial_df.timepoint==i_timepoint) & 
                                   (trial_df.side==0)]['likelihood_signal'].values[0];
                
                pnoise1 = trial_df[(trial_df.timepoint==i_timepoint) & 
                                   (trial_df.side==1)]['likelihood_noise'].values[0];
                pnoise0 = trial_df[(trial_df.timepoint==i_timepoint) & 
                                   (trial_df.side==0)]['likelihood_noise'].values[0];
                
                p1 = LLR2p(LLR1); # the probability that side==1
                
                LLR_present = LLR_present + \
                np.log(p1*psignal1*pnoise0 + (1-p1)*pnoise1*psignal0) - \
                np.log(pnoise0*pnoise1)
                
                LLR1 = LLR1+sample1-sample0;

                trial_LLR += [LLR_present,LLR_present]

            trial_decision = [int(LLR_present>0)]*24
            trial_confidence = [LLR2pcorrect(LLR_present)]*24     

            exp_LLR = exp_LLR + trial_LLR;
            exp_decision = exp_decision+trial_decision;
            exp_confidence = exp_confidence+trial_confidence;

        self.df['LLR'] = exp_LLR;
        self.df['decision'] = exp_decision;
        self.df['confidence'] = exp_confidence;
        
    def save(self,file_name):
        self.df.to_csv(file_name);

In [6]:
np.random.seed(1)

uv_det=DetectionModel([0,0.5],[1,1],2)
uv_det.runModel(10000,12)
uv_det.save(path.join(
        '..','simulations','equal_variance','detection.csv'))

In [9]:
uv.df.correct.mean()

0.7068